## Upload the provided kaggle.json file 

In [1]:
from google.colab import files #Go to your kaggle account & click on kaggle new api token, kaggle.json will get downloaded
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"musfirahfayyaz","key":"e4fff183de9ea6ffe3059a27dee1680b"}'}

In [2]:
!pip install -q kaggle #installing kaggle libraries

In [3]:
!mkdir -p ~/.kaggle #making directory of kaggle

In [4]:
!cp kaggle.json ~/.kaggle/ #putting kaggle.json in this directory



## Downloads the appliance dataset from kaggle. it contains 200 annotated  images of refrigerator and water dispenser
 

In [6]:
!kaggle datasets download -d musfirahfayyaz/appliance #from kaggle,go to your dataset copy api cmd
#exclamation mark is used in linux OS

appliance.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip /content/appliance.zip -d /content/data #unzip this file in content/data


Archive:  /content/appliance.zip
  inflating: /content/data/annotated/IMG_20220416_140637_Burst01.jpg  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst01.txt  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst02.jpg  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst02.txt  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst03.jpg  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst03.txt  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst04.jpg  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst04.txt  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst05.jpg  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst05.txt  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst06.jpg  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst06.txt  
  inflating: /content/data/annotated/IMG_20220416_140637_Burst07.jpg  
  inflating: /content/data/annotated/IMG_202

## Making training , testing and validation directories 

In [8]:
!mkdir train #these folders will be required for yolov5 training #70/30 normally
!mkdir train/images
!mkdir train/labels
!mkdir val
!mkdir val/images
!mkdir val/labels

## Loading all the files   and separating labels and images 

In [9]:
from os import walk
import matplotlib.pyplot as plt
path="/content/data/annotated" #path where files are =/content/data/...
filenames2 = next(walk(path), (None, None, []))[2]
path=path+"/"#/content/yolov5/data

In [10]:
filenames2 #un sorted rightnow

['IMG_20220416_140819_Burst10.jpg',
 'IMG_20220416_140913_Burst10.txt',
 'IMG_20220416_140819_Burst20.jpg',
 'IMG_20220416_140913_Burst07.txt',
 'IMG_20220416_140637_Burst08.jpg',
 'IMG_20220416_140819_Burst14.txt',
 'IMG_20220416_140913_Burst09.jpg',
 'IMG_20220416_140913_Burst04.txt',
 'IMG_20220416_140703_Burst08.jpg',
 'IMG_20220416_140917_Burst04.txt',
 'IMG_20220416_140637_Burst05.jpg',
 'IMG_20220416_140922_Burst07.txt',
 'IMG_20220416_140637_Burst14.jpg',
 'IMG_20220416_140901_Burst06.jpg',
 'IMG_20220416_140819_Burst12.txt',
 'IMG_20220416_140922_Burst20.txt',
 'IMG_20220416_140819_Burst19.txt',
 'IMG_20220416_140703_Burst20.txt',
 'IMG_20220416_140819_Burst19.jpg',
 'IMG_20220416_140913_Burst07.jpg',
 'IMG_20220416_140819_Burst03.jpg',
 'IMG_20220416_140905_Burst08.txt',
 'IMG_20220416_140637_Burst04.txt',
 'IMG_20220416_140913_Burst12.jpg',
 'IMG_20220416_140703_Burst07.txt',
 'IMG_20220416_140905_Burst01.jpg',
 'IMG_20220416_140917_Burst17.txt',
 'IMG_20220416_140819_Burst1

In [11]:
filenames2.sort() #sorted now
filenames2

['IMG_20220416_140637_Burst01.jpg',
 'IMG_20220416_140637_Burst01.txt',
 'IMG_20220416_140637_Burst02.jpg',
 'IMG_20220416_140637_Burst02.txt',
 'IMG_20220416_140637_Burst03.jpg',
 'IMG_20220416_140637_Burst03.txt',
 'IMG_20220416_140637_Burst04.jpg',
 'IMG_20220416_140637_Burst04.txt',
 'IMG_20220416_140637_Burst05.jpg',
 'IMG_20220416_140637_Burst05.txt',
 'IMG_20220416_140637_Burst06.jpg',
 'IMG_20220416_140637_Burst06.txt',
 'IMG_20220416_140637_Burst07.jpg',
 'IMG_20220416_140637_Burst07.txt',
 'IMG_20220416_140637_Burst08.jpg',
 'IMG_20220416_140637_Burst08.txt',
 'IMG_20220416_140637_Burst09.jpg',
 'IMG_20220416_140637_Burst09.txt',
 'IMG_20220416_140637_Burst10.jpg',
 'IMG_20220416_140637_Burst10.txt',
 'IMG_20220416_140637_Burst11.jpg',
 'IMG_20220416_140637_Burst11.txt',
 'IMG_20220416_140637_Burst12.jpg',
 'IMG_20220416_140637_Burst12.txt',
 'IMG_20220416_140637_Burst13.jpg',
 'IMG_20220416_140637_Burst13.txt',
 'IMG_20220416_140637_Burst14.jpg',
 'IMG_20220416_140637_Burst1

In [12]:
label=[i for i in filenames2 if i.endswith('.txt')] #.txt will get saved in label
images=[i for i in filenames2 if i.endswith('.jpg')] #.jpg will get saved in images

In [13]:
label1=[path + s for s in label] #path will be added before each file's name
images1=[path + s for s in images]


## Splitting the data into random training and testing. Rememeber 'input' are images and 'target' or 'y' are labels 

In [14]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #33% testing, 67% training
X_train, X_test, y_train, y_test = train_test_split(images1, label1, test_size=0.33, random_state=42)


# Copying the files into respective folders

In [15]:
import shutil

dst="/content/train/images"
for i in X_train:
    src=i
    shutil.copy2(src, dst)


In [16]:
dst="/content/val/images"
for i in X_test:
    src=i
    shutil.copy2(src, dst)


In [17]:
dst="/content/train/labels"
for i in y_train:
    src=i
    shutil.copy2(src, dst)

In [18]:
dst="/content/val/labels"
for i in y_test:
    src=i
    shutil.copy2(src, dst)
#up till now data preparaation has been done

##  Clonning YoloV5 and installing requirements 

In [19]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -r yolov5/requirements.txt  # install dependencies
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12875, done.
remote: Total 12875 (delta 0), reused 0 (delta 0), pack-reused 12875
Receiving objects: 100% (12875/12875), 11.84 MiB | 24.65 MiB/s, done.
Resolving deltas: 100% (8947/8947), done.
     |████████████████████████████████| 596 kB 5.2 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
/content/yolov5


# Training the data on appliance data. Upload appliance.yaml in the folder /content/yolov5/data

In [20]:
!python3 train.py --img 640 --batch 8 --epochs 25 --data appliance.yaml --weights yolov5s.pt --cache 

#train.py is an script,through which we train a model #img dimension #how many imgs in one batch #yolov5s.pt is a pretrained weight(smallest)
#so the accuracy is 99.5% for 25 epochs

train: weights=yolov5s.pt, cfg=, data=appliance.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=25, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-140-g3f3852e torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0

# Running the trained model on input data to generate a video with bounding box.

In [21]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --conf 0.4 --source /content/video.mp4 

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/video.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-140-g3f3852e torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/1791) /content/video.mp4: 352x640 1 Water_Dispenser, Done. (0.026s)
video 1/1 (2/1791) /content/video.mp4: 352x640 1 Water_Dispenser, Done. (0.026s)
video 1/1 (3/1791) /content/video.mp4: 352x640 1 Water_Dispenser, Done. (0.026s)
video 1/1 (4/1791) /content/video.mp4: 352x640 1 Water_Dispenser, Done. (0.026s)
video 1/1 (5/1791) /content/video.m

# Note the (best.pt)output video will be available in /content/yolov5/runs/detect/exp folder 